In [4]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_openai langchain_google_genai python-dotenv

In [3]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain-functional-api"

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp")

## Poem Flow

### 1. Synchronous

In [38]:
import os
from random import randint
from langgraph.func import entrypoint, task
from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize the AI Model (Poem Generation)
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp")

@task
def generate_sentence_count() -> int:
    """Generate a random sentence count for the poem."""
    return randint(1, 5)

@task
def generate_poem(sentence_count: int) -> str:
    """Generate a poem based on the sentence count using the AI model."""
    prompt = f"Write a beautiful and engaging poem about CrewAI with exactly {sentence_count} sentences."
    response = model.invoke(prompt)
    return response.content.strip()

@task
def save_poem(poem: str) -> str:
    """Save the poem to a file in a correct directory to avoid path errors."""
    output_dir = "output"
    os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists
    file_path = os.path.join(output_dir, "poem.txt")

    with open(file_path, "w", encoding="utf-8") as f:
        f.write(poem)

    return f"Poem saved successfully at {file_path}"

@entrypoint()
def run_workflow(input: str | None):
    """Workflow to generate and save a poem."""
    sentence_count = generate_sentence_count().result()
    poem = generate_poem(sentence_count).result()
    save_status = save_poem(poem).result()

    return {"sentence_count": sentence_count, "poem": poem, "status": save_status}

def stream():
    for event in run_workflow.stream(input=""):
        print(event)

In [39]:
stream()

{'generate_sentence_count': 2}
{'generate_poem': 'With digital minds as its crew, CrewAI embarks on quests, weaving intricate tasks into seamless, collaborative flows, a symphony of agents bringing innovation to life.\nThe team, like skilled navigators, charts courses through complex problems, crafting solutions with intelligence and grace, a testament to the power of collaborative AI.'}
{'save_poem': 'Poem saved successfully at output/poem.txt'}
{'run_workflow': {'sentence_count': 2, 'poem': 'With digital minds as its crew, CrewAI embarks on quests, weaving intricate tasks into seamless, collaborative flows, a symphony of agents bringing innovation to life.\nThe team, like skilled navigators, charts courses through complex problems, crafting solutions with intelligence and grace, a testament to the power of collaborative AI.', 'status': 'Poem saved successfully at output/poem.txt'}}


### 2. Asynchronous

In [41]:
import os
from random import randint
from langgraph.func import entrypoint, task
from langchain_google_genai import ChatGoogleGenerativeAI

@task
async def generate_sentence_count() -> int:
    """Generate a random sentence count for the poem."""
    return randint(1, 5)

@task
async def generate_poem(sentence_count: int) -> str:
    """Generate a poem based on the sentence count using the AI model."""
    prompt = f"Write a beautiful and engaging poem about AI Agents with exactly {sentence_count} sentences."
    response = await model.ainvoke(prompt)
    return response.content.strip()

@task
async def save_poem(poem: str) -> str:
    """Save the poem to a file in a correct directory to avoid path errors."""
    output_dir = "output"
    os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists
    file_path = os.path.join(output_dir, "poem.txt")

    with open(file_path, "w", encoding="utf-8") as f:
        f.write(poem)

    return f"Poem saved successfully at {file_path}"

@entrypoint()
async def run_workflow(input: str | None):
    """Workflow to generate and save a poem."""
    sentence_count = await generate_sentence_count()
    poem = await generate_poem(sentence_count)
    save_status = await save_poem(poem)

    return {"sentence_count": sentence_count, "poem": poem, "status": save_status}

async def stream():
    async for event in run_workflow.astream(input=""):
        print(event)

In [42]:
await stream()

{'generate_sentence_count': 1}
{'generate_poem': "Emerging from code's deep well, AI agents weave through the digital realm, their minds a symphony of algorithms, learning, adapting, and transforming, as they explore, connect, and create new possibilities, a testament to human ingenuity reaching beyond the bounds of flesh and bone."}
{'save_poem': 'Poem saved successfully at output/poem.txt'}
{'run_workflow': {'sentence_count': 1, 'poem': "Emerging from code's deep well, AI agents weave through the digital realm, their minds a symphony of algorithms, learning, adapting, and transforming, as they explore, connect, and create new possibilities, a testament to human ingenuity reaching beyond the bounds of flesh and bone.", 'status': 'Poem saved successfully at output/poem.txt'}}


----
----
> **The Concepts Below are Optional for AI-202 students go through.**
----

## Human in the Loop (Hello)

https://langchain-ai.github.io/langgraph/how-tos/wait-user-input-functional/?h=functiona

In [52]:
import time
import uuid

from langgraph.func import entrypoint, task
from langgraph.types import Command, interrupt


@task
def step_1(input_query):
    """Append bar."""
    return f"{input_query} bar"


@task
def human_feedback(input_query):
    """Append user input."""
    feedback = interrupt(f"Please provide feedback: {input_query}")
    return f"{input_query} {feedback}"


@task
def step_3(input_query):
    """Append qux."""
    return f"{input_query} qux"

In [54]:
from langgraph.checkpoint.memory import MemorySaver

checkpointer = MemorySaver()


@entrypoint(checkpointer=checkpointer)
def graph(input_query):
    result_1 = step_1(input_query).result()
    result_2 = human_feedback(result_1).result()
    result_3 = step_3(result_2).result()

    return result_3

In [57]:
thread_id = str(uuid.uuid4())

config = {
    "configurable": {
        "thread_id": thread_id
    }
}

for item in graph.stream("AI Agents", config):
    print(item)

{'step_1': 'AI Agents bar'}
{'__interrupt__': (Interrupt(value='Please provide feedback: AI Agents bar', resumable=True, ns=['graph:70dbb4c5-af34-bb93-2442-d0c244bca804', 'human_feedback:bd5aafb2-c018-828a-5481-eb7fbd30d55b'], when='during'),)}


In [58]:
# Get review from a user (e.g., via a UI)
# In this case, we're using a bool, but this can be any json-serializable value.
human_review = "proceed - send them to planner for this concept implementation"

for item in graph.stream(Command(resume=human_review), config):
    print(item)

{'human_feedback': 'AI Agents bar proceed - send them to planner for this concept implementation'}
{'step_3': 'AI Agents bar proceed - send them to planner for this concept implementation qux'}
{'graph': 'AI Agents bar proceed - send them to planner for this concept implementation qux'}


## Persistance (Short Term Memory)

https://langchain-ai.github.io/langgraph/how-tos/persistence-functional/?h=functiona

https://langchain-ai.github.io/langgraph/how-tos/cross-thread-persistence-functional/?h=functiona

In [45]:
import time
import uuid
from langgraph.func import entrypoint, task
from langgraph.checkpoint.memory import MemorySaver

# Setting up persistence
checkpointer = MemorySaver()

@task
def generate_city(country: str) -> str:
    """Generate a random city using an LLM call."""

    print("Starting flow")
    response = model.invoke(f"Return the name of a random city in the {country}.")
    random_city = response.content
    print(f"Random City: {random_city}")
    return random_city

@task
def generate_fun_fact(city: str) -> str:
    """Generate a fun fact about the given city."""

    response = model.invoke(f"Tell me a fun fact about {city}")
    fun_fact = response.content
    return fun_fact

@entrypoint(checkpointer=checkpointer)
def main_workflow(country: str) -> dict:
    """Main workflow that generates a random city and fetches a fun fact about it."""
    city = generate_city(country).result()
    fact = generate_fun_fact(city).result()

    return {"city": city, "fun_fact": fact, "country": country}


thread_id = str(uuid.uuid4())

config = {
    "configurable": {
        "thread_id": thread_id
    }
}

In [46]:
for result in main_workflow.stream("Pakistan", config):
  print(f"Generated fun fact: {result}")

Starting flow
Random City: Okay, here's a random city in Pakistan:

**Faisalabad**
Generated fun fact: {'generate_city': "Okay, here's a random city in Pakistan:\n\n**Faisalabad**"}
Generated fun fact: {'generate_fun_fact': 'Okay, here\'s a fun fact about Faisalabad, Pakistan:\n\n**Faisalabad is known as the "Manchester of Pakistan" due to its significant contribution to the country\'s textile industry.** Just like Manchester in England was a major hub for textiles during the Industrial Revolution, Faisalabad is a powerhouse for textile production in Pakistan, with numerous mills and factories dedicated to spinning, weaving, and processing fabrics.'}
Generated fun fact: {'main_workflow': {'city': "Okay, here's a random city in Pakistan:\n\n**Faisalabad**", 'fun_fact': 'Okay, here\'s a fun fact about Faisalabad, Pakistan:\n\n**Faisalabad is known as the "Manchester of Pakistan" due to its significant contribution to the country\'s textile industry.** Just like Manchester in England was 

In [47]:
main_workflow.get_state(config)

StateSnapshot(values={'city': "Okay, here's a random city in Pakistan:\n\n**Faisalabad**", 'fun_fact': 'Okay, here\'s a fun fact about Faisalabad, Pakistan:\n\n**Faisalabad is known as the "Manchester of Pakistan" due to its significant contribution to the country\'s textile industry.** Just like Manchester in England was a major hub for textiles during the Industrial Revolution, Faisalabad is a powerhouse for textile production in Pakistan, with numerous mills and factories dedicated to spinning, weaving, and processing fabrics.', 'country': 'Pakistan'}, next=(), config={'configurable': {'thread_id': 'd6059393-0643-4560-9670-f602016d8055', 'checkpoint_ns': '', 'checkpoint_id': '1efe2385-8b6c-6897-8000-f15a44655627'}}, metadata={'source': 'loop', 'writes': {'main_workflow': {'city': "Okay, here's a random city in Pakistan:\n\n**Faisalabad**", 'fun_fact': 'Okay, here\'s a fun fact about Faisalabad, Pakistan:\n\n**Faisalabad is known as the "Manchester of Pakistan" due to its significant

In [48]:
result2 = main_workflow.invoke("cat", config)
print(f"Generated fun fact: {result2}")

Starting flow
Random City: Okay, I understand! However, there seems to be a misunderstanding. You asked for the name of a random city "in the cat." Cats are animals, and they don't have cities. 

Perhaps you meant something else? Here are some possibilities and how I can help:

**Possible Interpretations and How I Can Help:**

1. **Random City in the World:** If you just want a random city from anywhere, I can easily do that! 
   * **Example:** "Okay, here's a random city: **Tokyo**."

2. **Random City in a Specific Country/Region:** If you're thinking of a specific place, like a country or continent, I can pick a random city from that area.
   * **Example:** "Give me a random city in Europe." then I would say something like "Okay, here's a random city in Europe: **Rome**."

3. **A City With a Cat Theme:** Perhaps you were looking for a city that has some kind of cat-related association (a cat cafe, cat statues, etc.)? While I can't guarantee a perfect "cat city", I could try my best t

In [49]:
main_workflow.get_state(config)

StateSnapshot(values={'city': 'Okay, I understand! However, there seems to be a misunderstanding. You asked for the name of a random city "in the cat." Cats are animals, and they don\'t have cities. \n\nPerhaps you meant something else? Here are some possibilities and how I can help:\n\n**Possible Interpretations and How I Can Help:**\n\n1. **Random City in the World:** If you just want a random city from anywhere, I can easily do that! \n   * **Example:** "Okay, here\'s a random city: **Tokyo**."\n\n2. **Random City in a Specific Country/Region:** If you\'re thinking of a specific place, like a country or continent, I can pick a random city from that area.\n   * **Example:** "Give me a random city in Europe." then I would say something like "Okay, here\'s a random city in Europe: **Rome**."\n\n3. **A City With a Cat Theme:** Perhaps you were looking for a city that has some kind of cat-related association (a cat cafe, cat statues, etc.)? While I can\'t guarantee a perfect "cat city", 

In [50]:
async for result in main_workflow.astream_events("Pakistan", config, version="v2"):
  print(f"Generated fun fact: {result}")

Generated fun fact: {'event': 'on_chain_start', 'data': {'input': 'Pakistan'}, 'name': 'LangGraph', 'tags': [], 'run_id': 'f2ab0f1b-77ed-408e-bb14-df4da93c6ea0', 'metadata': {'thread_id': 'd6059393-0643-4560-9670-f602016d8055'}, 'parent_ids': []}
Generated fun fact: {'event': 'on_chain_start', 'data': {'input': 'Pakistan'}, 'name': 'main_workflow', 'tags': ['graph:step:4'], 'run_id': 'cdcc156c-ef1b-4cbb-bfaa-66b731e75312', 'metadata': {'thread_id': 'd6059393-0643-4560-9670-f602016d8055', 'langgraph_step': 4, 'langgraph_node': 'main_workflow', 'langgraph_triggers': ['__start__'], 'langgraph_path': ('__pregel_pull', 'main_workflow'), 'langgraph_checkpoint_ns': 'main_workflow:96ae4f09-afd5-10c5-fed7-a2d21e4555c3'}, 'parent_ids': ['f2ab0f1b-77ed-408e-bb14-df4da93c6ea0']}
Generated fun fact: {'event': 'on_chain_start', 'data': {'input': {'country': 'Pakistan'}}, 'name': 'generate_city', 'tags': ['seq:step:1'], 'run_id': '0e57485d-28a6-4dd0-a1b5-82f37f45e687', 'metadata': {'thread_id': 'd605